In [2]:
!pip install rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, Polygon
import multiprocessing
from tqdm import tqdm
from rtree import index

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode', 'geometry'])

# 標高データの空間結合

In [6]:
df_elevation = dd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv', usecols=['X', 'Y', 'hyoukouchi'])

In [ ]:
#df_elevation = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv', usecols=['X', 'Y', 'hyoukouchi'])

In [7]:
# # カラムを絞る
# df_elevation = df_elevation[['X', 'Y', 'hyoukouchi']]

# カラムの名称を変更
df_elevation = df_elevation.rename(columns={'X': 'longitude', 'Y': 'latitude'})


# Pointオブジェクトの作成
geometry = df_elevation.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1, meta=('geometry', 'object')).compute()


# GeoPandasデータフレームを作成
gdf_elevation = gpd.GeoDataFrame(df_elevation.compute(), geometry=geometry)

#gdf_elevationに座標系はついていないので付与
if gdf_elevation.crs is None:
	gdf_elevation.set_crs(epsg=4326, inplace=True)

# 不要なメモリを解放
del df_elevation, geometry

# 標高のデータを結合
gdf_osyu = gpd.sjoin(gdf_osyu_100m_mesh, gdf_elevation, how='left', op='contains')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
gdf_osyu

,Meshcode,geometry,index_right,longitude,latitude,hyoukouchi
0,5840378485,"POLYGON ((140.93125 38.99000, 140.93125 38.990...",205202.0,140.932278,38.990611,395.9
0,5840378485,"POLYGON ((140.93125 38.99000, 140.93125 38.990...",205203.0,140.932389,38.990611,396.5
0,5840378485,"POLYGON ((140.93125 38.99000, 140.93125 38.990...",205185.0,140.932278,38.990722,396.3
0,5840378485,"POLYGON ((140.93125 38.99000, 140.93125 38.990...",205201.0,140.932167,38.990611,395.2
0,5840378485,"POLYGON ((140.93125 38.99000, 140.93125 38.990...",205184.0,140.932167,38.990722,396.1
...,...,...,...,...,...,...
101385,5841730010,"POLYGON ((141.37500 39.25083, 141.37500 39.251...",153368.0,141.375167,39.251278,618.4
101385,5841730010,"POLYGON ((141.37500 39.25083, 141.37500 39.251...",153369.0,141.375278,39.251278,619.1
101385,5841730010,"POLYGON ((141.37500 39.25083, 141.37500 39.251...",153367.0,141.375056,39.251278,617.3
101385,5841730010,"POLYGON ((141.37500 39.25083, 141.37500 39.251...",153659.0,141.375056,39.251389,615.7


In [9]:
#標高の値をグループ演算で平均、最大、最低を求める
gdf_elevation = gdf_osyu.groupby('Meshcode').agg({'hyoukouchi':['mean', 'max', 'min']}).reset_index()

In [10]:
gdf_elevation

Meshcode  hyoukouchi              
                          mean    max    min
0       5840378485  394.293750  397.2  389.6
1       5840378486  378.428000  395.4  363.1
2       5840378487  371.850000  374.1  369.3
3       5840378495  383.426667  395.4  359.1
4       5840378496  359.411688  386.8  333.2
...            ...         ...    ...    ...
101381  5841728323  476.184615  487.0  471.8
101382  5841728324  477.800000  480.7  475.5
101383  5841730000  620.780952  634.0  600.3
101384  5841730001  644.250000  655.7  634.1
101385  5841730010  621.613636  625.7  615.7

[101386 rows x 4 columns]

In [11]:
# カラム名を変更
gdf_elevation.columns = ['Meshcode', 'hyoukouchi_mean', 'hyoukouchi_max', 'hyoukouchi_min']

In [12]:
#標高の最大-最小のカラムを追加
gdf_elevation['hyoukouchi_max-hyoukouchi_min'] = gdf_elevation['hyoukouchi_max'] - gdf_elevation['hyoukouchi_min']

In [13]:
gdf_elevation[gdf_elevation['hyoukouchi_max-hyoukouchi_min'] < 0]

,Meshcode,hyoukouchi_mean,hyoukouchi_max,hyoukouchi_min,hyoukouchi_max-hyoukouchi_min


In [14]:
gdf_elevation.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　標高/奥州市_標高.csv')